In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
df = pd.read_json("arxivData.json")
df.head()

In [ ]:
df = df[['summary', 'tag', 'title']]
df.head()

In [ ]:
df.info()

In [ ]:
df['summary'][0]

In [ ]:

from ast import literal_eval

l = []
for tag in df['tag']:
    l.append(literal_eval(tag))
    
l[0][0]['term']
    


In [ ]:
len(df)

# Gets the unique tags

In [ ]:
unique_tags = []

In [ ]:
for i in range(len(l[0])):
    if l[0][i]['term'] not in unique_tags:
        unique_tags.append(l[0][i]['term'])

In [ ]:
unique_tags

# Create a list of lists of tags

In [ ]:
all_tags = []
for point in range(len(df)): 
    for i in range(len(l[point])):
        all_tags.append(l[point][i]['term'])

len(set(all_tags))

### 2293 unique tags as shown above

In [ ]:
# tag_counts = {}
import collections

tag_counts = collections.Counter(all_tags)
tag_counts

sorted_tag_counts = sorted(tag_counts.items(), key=lambda kv: kv[1], reverse=True)
#sorted_tag_counts


In [ ]:
for k, v in sorted_tag_counts:
    if v > 900:
        print(k)

## Create a dictionary of most relevant tags

In [ ]:
most_relevant_tags = {}
for k, v in sorted_tag_counts:
    if v > 900:
        print(k)
        most_relevant_tags[k] = v
most_relevant_tags

In [ ]:
tdf = pd.DataFrame(data=most_relevant_tags, index=[0])


In [ ]:
tdf.head()

In [ ]:
sns.barplot(data=tdf)

# Create list of most relevant label vectors

In [ ]:
list_of_all_tags = []
for point in range(len(df)): 
    temp_list = []
    
    for i in range(len(l[point])):
        temp_list.append(l[point][i]['term'])
    list_of_all_tags.append(temp_list)

list_of_all_tags[:10]

In [ ]:
mr=[ ] # list of tags [[ ..41000 entries ..], [], [], ..., []] .. 41000 x 9 matrix

for mr_tag in most_relevant_tags:
    temp = []
    for each_tag_list in list_of_all_tags:
        if mr_tag in each_tag_list:
            temp.append(1)
        else:
            temp.append(0)
    mr.append(temp)

# mr
np.array(mr).shape

# Eliminate text instances with no Top-9 tags

Checking if there are text instances for which none of Top-9 tags exist.

In [ ]:
mr_transpose = list(map(list, zip(*mr)))
index_list = []

for i in range(len(mr_transpose)):
    if mr_transpose[i].count(0) == 9:
        index_list.append(i)

index_list

No such text instances found, so no need for any elimination.

# Create final Dataframe

Join Abstract and title

In [ ]:
df['text'] = df[['summary', 'title']].apply(lambda x: ' '.join(x), axis=1)

In [ ]:
df['text'][0]

Create list of texts

In [ ]:
text_list = []
for i in range(41000):
    text_list.append(df['text'][i])

Create new Dataframe

In [ ]:
data = list(map(list, zip(*[text_list, mr[0], mr[1], mr[2], mr[3], mr[4], mr[5], mr[6], mr[7], mr[8]])))
final_df = pd.DataFrame(data)
final_df

Exporting to CSV for further use:

In [ ]:
final_df.to_csv('processed_data.csv')

# Creating dataframe for Label Poset

In [ ]:
import operator

mr_list_str = []
for i in range(41000):
    mr_list_str.append(''.join(map(str, mr_transpose[i])))

classes = list(range(len(mr_list_str)))    
sorted_mr_list_str = sorted(list(set(mr_list_str)))    
str_dictionary = dict(zip(sorted_mr_list_str, classes))
print(str_dictionary)    
#text_dictionary = dict(zip(text_list, mr_list_str)) #Mapping each text combination to corresponding most relevant tag string
#sorted_text_dictionary = dict(sorted(text_dictionary.items(), key=operator.itemgetter(1))) #Sorting text dictionary based on most relevant tag string

#for key, value in sorted_text_dictionary.items():
#    sorted_text_dictionary[key] = str_dictionary[value]

Creating Pandas Dataframe and Processed CSV from dictionary

In [ ]:
lp_df = pd.DataFrame(list(zip(list(sorted_text_dictionary.keys()), list(sorted_text_dictionary.values()))),
              columns = ['Abstract','Class'])
lp_df.to_csv('processed_data_lp.csv')